In [34]:
import pandas as pd
import os
import sys
module_path = os.path.abspath(os.path.join('/Users/balbi/Downloads/pyMorfologik-master'))
if module_path not in sys.path:
    sys.path.append(module_path)
from pymorfologik import Morfologik
from pymorfologik.parsing import ListParser
import numpy as np
from __future__ import print_function
import string
from multiprocessing import cpu_count, Pool
import dask.dataframe as dd
from dask.multiprocessing import get

In [35]:
def lemmatisation(text):
    parser = ListParser()
    stemmer = Morfologik()
    stemming = stemmer.stem([text], parser)
    words_list = list()
    for s in stemming:
        for i in s:
            for l in i:
                if(len(l) > 1):
                    words_list.append(l)
    return words_list

In [36]:
emotions = pd.read_csv('emotionsTable-with-neutralEmotion.csv')
def detectingEmotionsInText(words_list):
    numberOfWords = len(words_list)
    numberOfWordsFound = 0
    result = 0
    if(numberOfWords > 0):
        emotionsCounter = 0
        for el in words_list: 
            if((emotions['word'] == el)).any():
                emotionsCounter += float(emotions.loc[emotions['word'] == el]['sentiment'].mean())  
                numberOfWordsFound = numberOfWordsFound + 1
        result = emotionsCounter/numberOfWords
    return [result, numberOfWordsFound]

In [37]:
def getEmotionStatusOfText(result):
    if(result < 0):
        return 'negative'
    elif(result > 0):
        return 'positive'
    else:
        return 'neutral'

In [39]:
data1 = pd.read_csv('FilmWeb-commentsRates.csv')
data2 = pd.read_csv('FilmWeb-commentsRates-moreMovies.csv')
result = data1.append(data2)

result = result.drop_duplicates(['comment'], keep='last')
result = result.reset_index()
#result = result.drop('level_0', 1)

In [18]:
%%time
import sys
import re
reload(sys)
sys.setdefaultencoding('utf-8')

def calculateOneRow(index,comment, emotion, rate):
    lemmatisation_text = re.sub("[!#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~']", '', comment)
    words_list = lemmatisation(lemmatisation_text)
    [result, number] = detectingEmotionsInText(words_list)
    result_emotion = getEmotionStatusOfText(result)
    return {"text": comment, 'words': len(comment.split()), "lemmatisation": words_list, 'wordsAfterLemmatisation': len(words_list), 'wordsFound': number, "resultOfDetecting": result, "detectedEmotion": result_emotion, 'emotion': emotion, 'rate': rate}

def parallelize(data, func):
    print (partitions)
    data_split = np.array_split(data, partitions)
    pool = Pool(cores)
    mapResult = pool.map(func, data_split)
    data = pd.concat(mapResult)
    pool.close()
    pool.join()
    return data

cores = cpu_count()
partitions = cores

ddata = dd.from_pandas(result, npartitions=partitions)

result = ddata.map_partitions(lambda df: df.apply((lambda row: calculateOneRow(*row)), axis=1)).compute(get=get)
result_df = pd.DataFrame()
for o in result:
    result_df = result_df.append(o, ignore_index=True)
    
result_df.to_csv('FilmWeb-comments-analysis-neutralemotion-results-3.csv', encoding='utf-8', index=False) 

,index,comment,emotion,rate
0,0,"Strzał w kolano dla wszystkich facetów Starałem się przeczytać książkę. Uwierzcie mi, naprawdę się starałem. Jednak nie wytrzymałem wyidealizowanego opisu postaci Greya. Młody, przystojny, milioner, demon seksu, właściciel ogromnej korporacji i do tego z mnóstwem wolnego czasu, który w całości poświęca kobiecie. Powinien ...",positive,6
1,1,"Śmierdzące gówno w srebrnym celofaniku... Włączam ja sobie to filmidło, oczywiście z wielkimi obawami. No i nie zawiodłem się bo oczekiwania miałem takie, jakbym korzystał z kibla postawionego na środku opuszczonej wsi. Mierne aktorstwo, początkujący lub pozbawieni talentu tzw. ""aktorzy"", którzy mogą być idolami najwyżej ...",negative,2
2,2,"Całkowite zaskoczenie Od razu zaznaczę, że nie czytałam książki. Do filmu podchodziłam delikatnie mówiąc sceptycznie, a tak naprawdę chciałam go obejrzeć tylko po to, by móc go skrytykować. Nastawiałam się, że jest to film dla nastolatek, epatujący wulgarnym seksem i żadnemu dorosłemu człowiekowi ze zdrowym i dojrzałym po...",positive,6
3,3,"""Boże, zaraz się zrzygam"".. Słowa Koroniewskiej chyba najlepiej opisują to dzieło.Wszystko w tym filmie było złe; ""fabuła"", postaci, aktorzy, najgorsze na świecie dialogi. Totalny dramat, gorszy nawet od ""Zmierzchu"". Plus za realizm typu prawdziwe cycki, włosy łonowe, bujne owłosienie na udach kobiety i pryszcze na klacie...",negative,1
4,4,"Jak ""Zmierzch"" nie wiem czy ktos to zauważył ale mnie zalatuje Zmierzchem. Dialogi takie jak ""powinnaś sie trzymać ode mnie z daleka"", sama gra aktorów. Dopiero widziałam 15 min filmu a juz mnie roznosi ta zmierzchowa oschłość...",positive,8
5,6,"Może gdyby ktoś próbował uratować ten film przed dialogami i chłopięcym Greyem... Dialogi były tak złe jak tylko mogły. Oglądaliśmy ten film w łóżku z narzeczonym i przez większość filmu skręcaliśmy się ze śmiechu przy każdym dialogu (najlepszy był fragment: ""I don't make love. I f..k. Hard.""). To było naprawdę złe. Dakot...",negative,2
6,7,"Film dla ""ajfonowych"" lemingów z ujemnym IQ. Ten film nie jest filmem.Darwin nie miał racji...ewolucja ukierunkowana jest w stronę małpy a nie na odwrót.Niskie pobudki,wyzelowane z wartości życie i zapampersione marzenia.Duplikat staje się powtórką remake'ów.",negative,1
7,8,"""Porno"" dla ubogich (mieszczek) Dla młodszych nastolatek i znudzonych gospodyń domowych na podobnym poziomie rozwoju intelektualnego i emocjonalnego.Filmidło, romansidło jadące na kilku chamskich zasadach.Że każda potwora marzy o rycerzu w lśniącej zbroi, czyli... kimkolwiek bogatym (a która nie puści się za dobry samochó...",negative,4
8,9,"Słabo Gra aktorska na poziomie polskiej reprezentacji piłki nożnej. Drewno. Ogólnie to naiwny film o krzywdzącej miłości, nic specjalnego.",negative,3
9,10,"To najbardziej antyfeministyczna rzecz w dziejach świata i to nawet nie dlatego że on ją napier*dala po dupie czy po czym tam. Rzecz w tym że Christianek jest w tym filmie wyidealizowaną studnią bez dna (Adonis z sześciopakiem, rucha jak Zeus, forsy ma jak wujek Sknerus, mówi po Francusku, gra na fortepianie, pilotuje hel...",negative,1
